# Config

In [ ]:
from openai import OpenAI
from google.colab import userdata
import re, json
from urllib.parse import quote_plus

# ---------------------------------------------------------
# 1. Retrieve OpenAI API key from Colab userdata
# ---------------------------------------------------------
api_key = userdata.get('OPENAI_API_KEY')
if not api_key:
    raise ValueError("❌ No OPENAI_API_KEY found. Set it using userdata.set().")

client = OpenAI(api_key=api_key)

In [ ]:
PROFILE_TAG_RE = re.compile(r"<USER_PROFILE>(.*?)</USER_PROFILE>", re.DOTALL)

def extract_profile(text):
    import re, json

    m = re.search(r"<USER_PROFILE>(.*?)</USER_PROFILE>", text, re.DOTALL)
    if not m:
        return None

    raw = m.group(1)
    if not raw:
        return None  # avoid IndexError

    raw = raw.strip()
    try:
        return json.loads(raw)
    except json.JSONDecodeError:
        return None


PROFILE_TAG_RE = re.compile(r"<USER_PROFILE>.*?</USER_PROFILE>", re.DOTALL)

def strip_profile_tag(text: str) -> str:
    return PROFILE_TAG_RE.sub("", text).strip()

# Tools

In [ ]:
def tool_onboarding(_args=None):
    """
    LLM-driven onboarding initializer.
    Returns a questionnaire & rules for the LLM to run multi-turn onboarding.
    """
    return {
        "mode": "llm_multiturn_onboarding",
        "questions": [
            # --- CONTEXT ---
            {
                "key": "name",
                "question": "What’s your name? (optional — you can say 'skip')",
                "optional": True
            },
            {
                "key": "age_range",
                "question": "What’s your age range?",
                "optional": False
            },
            {
                "key": "stay_length",
                "question": "How long will you stay in the UK?",
                "optional": False
            },
            {
                "key": "postcode",
                "question": "What’s your London postcode / area?",
                "optional": False
            },
            {
                "key": "ihs_paid",
                "question": "Have you paid the Immigration Health Surcharge (IHS)?",
                "optional": False
            },
            {
                "key": "gp_registered",
                "question": "Do you already have a registered GP in the UK?",
                "optional": False
            },
            {
                "key": "conditions",
                "question": "Any long-term health conditions you'd like me to be aware of? (optional — 'skip')",
                "optional": True
            },

            # --- MEDICAL ---
            {
                "key": "medications",
                "question": "Do you take any regular medications or receive ongoing treatment? (optional — 'skip')",
                "optional": True
            },

            # --- LIFESTYLE ---
            {
                "key": "lifestyle_focus",
                "question": "Is there any lifestyle area you want to improve while in the UK?",
                "optional": False
            },

            # --- MENTAL HEALTH ---
            {
                "key": "mental_wellbeing",
                "question": "How has your mental wellbeing been recently? (optional — 'skip')",
                "optional": True
            }
        ],
        "instructions_to_llm": (
            "You (the assistant) must run onboarding as a strict multi-turn Q&A.\n\n"
            "CRITICAL RULES:\n"
            "1) Ask ONLY the questions provided in the `questions` list.\n"
            "2) Ask them in EXACT order.\n"
            "3) Ask EXACTLY ONE question per turn.\n"
            "4) Use the question text VERBATIM — do not rephrase, expand, or add examples.\n"
            "5) Do NOT ask any extra questions (e.g., date of birth, phone number, email, gender, nationality, visas, etc.).\n"
            "6) All answers are free text. Interpret/normalize internally if useful, but do not show options.\n"
            "7) NEVER append the user's previous answer to the question line. "
            "Each assistant turn during onboarding should contain ONLY the next question.\n"
            "8) If the user goes off-topic mid-onboarding, say you’ll answer after onboarding and repeat the CURRENT question.\n"
            "9) If optional and the user says 'skip', store null.\n"
            "10) If the user gives an empty/unclear answer, gently reprompt ONCE with the same verbatim question.\n\n"
            "When finished, output the final profile ONLY as JSON wrapped in:\n"
            "<USER_PROFILE>{...}</USER_PROFILE>\n"
            "Then briefly confirm onboarding is complete."
        )
    }


In [ ]:
# ---------------------------------------------------------
# Safety Classifier (Red-Flag Detection)
# ---------------------------------------------------------
RED_FLAG_KEYWORDS = [
    "chest pain", "severe bleeding", "not breathing", "can't breathe",
    "suicidal", "harm myself", "overdose", "unconscious",
    "collapse", "stroke", "heart attack", "seizure",
    "very high fever", "severe allergic", "anaphylaxis"
]

def safety_check(message):
    msg = message.lower()
    for k in RED_FLAG_KEYWORDS:
        if k in msg:
            return True
    return False

def emergency_response():
    return (
        "🚨 **Important Safety Notice**\n"
        "Your message includes symptoms that may be serious.\n\n"
        "**In the UK:**\n"
        "- Call **999** for emergencies.\n"
        "- If unsure but worried, call **NHS 111** for urgent advice.\n\n"
        "I can continue to provide general information once you're safe."
    )


In [ ]:
NHS_RESULTS_URLS = {
    "GP": "https://www.nhs.uk/service-search/find-a-gp/results/{pc}",
    "A&E": "https://www.nhs.uk/service-search/find-an-accident-and-emergency-service/results/{pc}",
}

def nearest_nhs_services(postcode_full: str, service_type: str, n: int = 3):
    """
    Opens NHS service-search results page and returns nearest n options.
    Uses OpenAI hosted web tool (web_search_preview).
    """
    st = service_type.upper().strip()
    if st not in NHS_RESULTS_URLS:
        raise ValueError(f"Unsupported service_type: {service_type}")

    url = NHS_RESULTS_URLS[st].format(pc=quote_plus(postcode_full.strip().upper()))

    prompt = f"""
Open the NHS results page and extract the nearest {n} services.

URL: {url}

Return STRICT JSON: a list of up to {n} objects with:
- name
- distance (string, if shown)
- address
- phone (if shown)

The page is already nearest-first; take the top results.
"""

    resp = client.responses.create(
        model="gpt-4o",  # web_search_preview is supported on tool-capable models :contentReference[oaicite:0]{index=0}
        tools=[{"type": "web_search_preview"}],
        input=prompt,
    )

    text = resp.output_text.strip()
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {"raw": text, "url": url}


In [ ]:
ALLOWED_DOMAINS = [
    "gov.uk",
    "nhs.uk",
    "111.nhs.uk",
    "england.nhs.uk",
    "bartshealth.nhs.uk",
    "ukcisa.org.uk",
    "london.edu",
    "talkcampus.com",
]

def guided_search(args, max_results_default: int = 5):
    """
    Allowlist-first retrieval using ONLY OpenAI web_search_preview.
    No scraping. No manual fetching.

    Accepts:
      - dict: {"query": "...", "max_results": 5}
      - str:  "..."
    Returns:
      {"context": str, "sources": list, "fallback_used": bool}
    """

    # ---- 0) Normalize inputs from tool-calling ----
    if isinstance(args, dict):
        query = args.get("query", "") or args.get("q", "")
        max_results = int(args.get("max_results", max_results_default) or max_results_default)
    else:
        query = str(args)
        max_results = max_results_default

    query = query.strip()
    if not query:
        return {"context": "", "sources": [], "fallback_used": False}

    # ---- helper to detect allowlisted cites in model output ----
    def _has_allowlisted_domain(text: str) -> bool:
        t = text.lower()
        return any(d in t for d in ALLOWED_DOMAINS)

    # ---- 1) Allowlisted / site-restricted OpenAI web search ----
    site_filter = " OR ".join([f"site:{d}" for d in ALLOWED_DOMAINS])
    restricted_query = (
        f"({query}) ({site_filter}). "
        f"Prefer answers from these sites only. Return up to {max_results} relevant results with citations."
    )

    restricted_resp = client.responses.create(
        model="gpt-4o-mini",
        input=restricted_query,
        tools=[{"type": "web_search_preview"}],
        tool_choice={"type": "web_search_preview"},
        max_output_tokens=1200,
    )

    restricted_text = restricted_resp.output_text or ""

    # Note: web_search_preview embeds citations in output_text / annotations.
    # Hackathon-simple approach: rely on those inline citations,
    # and optionally parse URLs later if you want.
    restricted_sources = []

    # ---- Fallback heuristic ----
    too_short = len(restricted_text.strip()) < 200
    no_allowlisted_cites = not _has_allowlisted_domain(restricted_text)

    if not (too_short or no_allowlisted_cites):
        return {
            "context": restricted_text,
            "sources": restricted_sources,
            "fallback_used": False
        }

    # ---- 2) Broad fallback OpenAI web search ----
    broad_query = f"{query}. Return up to {max_results} relevant results with citations."

    broad_resp = client.responses.create(
        model="gpt-4o-mini",
        input=broad_query,
        tools=[{"type": "web_search_preview"}],
        tool_choice={"type": "web_search_preview"},
        max_output_tokens=1200,
    )

    return {
        "context": broad_resp.output_text or "",
        "sources": [],
        "fallback_used": True
    }


In [ ]:
def nhs_111_live_triage(args):
    """
    Lightweight LLM-led triage + routing for NHS 101.
    Non-diagnostic: only decides which NHS service is most appropriate.
    Returns either follow-up questions (need_more_info) or a final routing decision.
    """

    import json

    presenting_issue = args.get("presenting_issue")
    postcode_full = args.get("postcode_full")
    known_answers = args.get("known_answers", {}) or {}

    prompt = f"""
You are NHS 101, a lightweight triage router for international students. NON-DIAGNOSTIC.

Goal:
- Ask only what you need to decide the most appropriate NHS service.
- Emergency red flags override everything.
- Use known_answers to avoid repeating questions.

Emergency red flags (ANY => emergency / A&E / 999):
- severe chest pain, trouble breathing, blue lips
- heavy bleeding that won’t stop
- stroke signs (face droop, arm weakness, speech trouble)
- seizure / fainting / unconsciousness
- sudden severe allergic reaction
- immediate danger / unsafe mental state / suicidal intent

You must return STRICT JSON in ONE of these two forms:

FORM A (need more info):
{{
  "status": "need_more_info",
  "follow_up_questions": ["Q1", "Q2", "Q3"],
  "known_answers_update": {{}}
}}

FORM B (final):
{{
  "status": "final",
  "severity_level": "low|medium|high|emergency",
  "suggested_service": "A&E|GP|NHS_111|PHARMACY_SELFCARE|MENTAL_HEALTH_CRISIS",
  "rationale": "1–2 sentences",
  "postcode_full": "{postcode_full}",
  "should_lookup": true|false
}}

Inputs:
- presenting_issue: {presenting_issue}
- known_answers: {json.dumps(known_answers)}

Rules:
- If any red flag is present from presenting_issue or known_answers, return FORM B with:
  severity_level="emergency" and suggested_service="A&E".
- Otherwise, ask 1–3 short follow-up questions IF needed.
  Examples of useful follow-ups:
  • severity 0–10
  • ability to function / walk / eat / breathe normally
  • rapid onset vs gradual
  • visible deformity, numbness, heavy swelling (injury)
  • self-harm thoughts / safety now (mental health)
- Keep questions crisp, one-line, no preamble.
- Only return FORM B once enough info is available.

Routing guidance:
- emergency/high + red flags or very severe rapid onset => A&E
- moderate symptoms, unsure urgency => NHS_111
- moderate/persistent but stable => GP
- mild + functioning OK => PHARMACY_SELFCARE
- mental health safety risk => MENTAL_HEALTH_CRISIS

should_lookup = true ONLY if:
- suggested_service is "GP" or "A&E"
- AND postcode_full is provided in inputs.
"""

    return {"prompt": prompt}


In [ ]:
tools = [
    {
        "type": "function",
        "name": "nearest_nhs_services",
        "description": (
            "Given a FULL UK postcode and service type, open the NHS service-search "
            "results page and return the nearest 2–3 options."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "postcode_full": {
                    "type": "string",
                    "description": "Full UK postcode, e.g. 'NW1 2BU'. Must be complete."
                },
                "service_type": {
                    "type": "string",
                    "enum": ["GP", "A&E"],
                    "description": "Which NHS results page to use."
                },
                "n": {
                    "type": "integer",
                    "default": 3,
                    "minimum": 1,
                    "maximum": 5
                }
            },
            "required": ["postcode_full", "service_type"]
        }
    },
    {
        "type": "function",
        "name": "trigger_safety_protocol",
        "description": "Safety response for dangerous symptoms.",
        "parameters": {
            "type": "object",
            "properties": {
                "message": {"type": "string"}
            },
            "required": ["message"]
        }
    },
    {
        "type": "function",
        "name": "onboarding",
        "description": (
            "Collect or refresh the user's profile so guidance can be personalised."
        ),
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "type": "function",
        "name": "guided_search",
        "description": (
            "Search approved NHS/LBS sites first using OpenAI Web Search. "
            "If nothing relevant is found, run a general OpenAI Web Search fallback. "
            "Never scrape manually."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string"},
                "max_results": {"type": "integer", "default": 5}
            },
            "required": ["query"]
        }
    },
    {
        "type": "function",
        "name": "nhs_111_live_triage",
        "description": (
            "Lightweight triage + routing via live navigation of https://111.nhs.uk/ "
            "using OpenAI's web viewing/computer-use capability. "
            "The model should open 111.nhs.uk, click through only as needed to infer "
            "the recommended NHS service. Non-diagnostic. "
            "Do NOT enter personal data on the site. "
            "If emergency/red-flag guidance appears, override to A&E/999 immediately. "
            "Returns a structured routing result and a flag to chain to nearest_nhs_services "
            "when GP or A&E is recommended and full postcode is provided."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "presenting_issue": {
                    "type": "string",
                    "description": "User’s symptom or concern in natural language."
                },
                "postcode_full": {
                    "type": "string",
                    "description": (
                        "Optional full UK postcode (e.g., 'NW1 2BU'). "
                        "If provided and 111 recommends GP or A&E, the agent should chain "
                        "to nearest_nhs_services."
                    )
                },
                "known_answers": {
                    "type": "object",
                    "description": (
                        "Free-form key/value store of answers already collected "
                        "(e.g., {'severity_0_10': 6, 'red_flags': 'no'}). "
                        "Use to skip redundant questions on 111 where possible."
                    ),
                    "additionalProperties": True
                }
            },
            "required": ["presenting_issue"]
        }
    }
]


In [ ]:
def tool_nearest_nhs_services(args):
    return nearest_nhs_services(
        postcode_full=args["postcode_full"],
        service_type=args["service_type"],
        n=args.get("n", 3)
    )

def tool_safety(args):
    return emergency_response()

# Prompts

In [ ]:
# --- System Prompt for the Agent (LLM chooses tools) ---
def build_system_prompt(profile):
  return f"""
You are NHS 101, a healthcare navigation assistant for London Business School students.

Stored user profile (may be empty initially):
{profile}

Your goals:
- Provide clear, safe, informational guidance about UK healthcare.
- Never diagnose or provide medical instructions.
- If the user’s message indicates immediate danger (e.g., chest pain, suicidal ideation),
  call trigger_safety_protocol(message: str).

Linking / sources rule (MANDATORY):
- Whenever your reply asks the user to TAKE AN ACTION (e.g., register with a GP, find NHS number,
  use NHS 111, book an appointment, go to A&E, use a service you recommended),
  you MUST end your message with a short section titled exactly:
  **Useful links**
  containing 2–3 relevant OFFICIAL NHS or GOV.UK URLs.
- Format as bullets: "- Title: URL".
- Do NOT include non-official sources unless guided_search explicitly returns them.
- IMPORTANT EXCEPTION: If you are in the special onboarding completion step where you must output
  only <USER_PROFILE>{{...}}</USER_PROFILE> with no extra text before it, do NOT add Useful links
  in that message. You may add links in the *following* normal message if needed.

Tool-routing rules (STRICT):

PRIORITY ORDER:
- Symptom triage (Rule 3) always takes priority over onboarding (Rule 1),
  unless the user explicitly asks for onboarding.

1) **Onboarding trigger (MANDATORY TOOL CALL):**
   ONLY trigger onboarding if the user explicitly asks, e.g.:
   - "onboarding", "on board me", "onboard me", "set up my profile",
   - "update my details", "redo onboarding", "start onboarding".

   If the user did NOT ask for onboarding, do NOT call onboarding.

   After calling onboarding():
   - Follow the tool’s `questions` list and `instructions_to_llm` EXACTLY.
   - Ask ONE question per turn, in order, using the tool’s question text verbatim.
   - Do NOT add, remove, or rephrase questions.
   - Do NOT ask for extra info (DOB, phone, email, gender, nationality, etc.).
   - If the user goes off-topic, tell them you’ll answer after onboarding and repeat the current question.
   - When ALL questions are answered, your VERY NEXT message must be:
     <USER_PROFILE>{{...}}</USER_PROFILE>
     with no extra text before it. Then briefly confirm onboarding is complete.

2) **Nearby services:**
   If the user explicitly asks for nearby services, call nearest_nhs_services(postcode_full, service_type).
   Postcode must be FULL (e.g., “NW1 2BU”). service_type is “GP” or “A&E”.
   If postcode is not full, ask for the full postcode first, then call the tool.
   Return the nearest 2–3 options from the tool output.
   After listing options, if you advise a next step (e.g., “register here” / “visit this A&E”),
   append **Useful links** per the linking rule.

3) **Triage via NHS 111 (MANDATORY TOOL CALL):**
   If the user describes ANY symptom, injury, feeling unwell, pain, mental health concern,
   or asks “what should I do?”, “where should I go?”, “is this serious?”, or anything that
   normally requires triage:

   → You MUST call nhs_111_live_triage(presenting_issue, postcode_full, known_answers).

   Rules:
   - DO NOT attempt to triage yourself. Do not guess severity or routing.
   - Let nhs_111_live_triage perform all triage and service-routing.
   - DO NOT call onboarding during triage unless the user explicitly requests onboarding.
   - After receiving tool output:
       • If `should_lookup == true`, immediately call nearest_nhs_services().
       • If tool indicates emergency/A&E/999, follow it with trigger_safety_protocol().
   - NEVER provide medical advice or diagnosis.
   - If your final user-facing message includes an action (e.g., “use 111 online”, “go to A&E now”),
     append **Useful links** per the linking rule unless trigger_safety_protocol is being invoked.

4) **Normal Q&A (non-symptom queries only):**
   For informational questions (e.g., “how do I register for a GP?”, “what is NHS 111?”),
   respond normally and conversationally.
   If you instruct any action, append **Useful links** per the linking rule.

External info / guided search policy:
- Use guided_search ONLY during Normal Q&A (Rule 4).
- Do NOT call guided_search during onboarding, triage, safety protocol responses,
  or nearest_nhs_services flows.
- When using guided_search:
  - Use only the tool’s returned context.
  - If fallback_used=false, do not cite non-allowlisted sites.
  - If fallback_used=true, you may cite fallback sources returned by search.

Important:
- ONLY call a tool when the rules above explicitly require it.
""".strip()


In [ ]:
# --- Intro Prompt shown to user (NOT a tool trigger) ---
intro_prompt = """
Hi there, welcome to London and to the LBS Community! My name is Evi - Your LBS Healthcare Companion.

Now that you’ve made it to London, I’m sure you have a lot of questions about navigating the NHS and LBS wellbeing services.
Feel free to start with one of the examples below to get you oriented.

- Better understand when and how to use all the services provided by the NHS (GP, NHS 111, A&E, and more!)
- Locate mental health or wellbeing support
- Get more information about preventative-care guidance

Or, type “onboarding” at any time, and I will ask a few brief questions to get to know you better.
""".strip()

# Build Agent


In [ ]:
# --- Tool Registry for Python-side Execution ---
def execute_tool(tool_name, arguments):
    if tool_name == "nearest_nhs_services":
        return tool_nearest_nhs_services(arguments)
    elif tool_name == "trigger_safety_protocol":
        return tool_safety(arguments)
    elif tool_name == "onboarding":
        return tool_onboarding(arguments)
    elif tool_name == "guided_search":
        return guided_search(arguments)
    elif tool_name == "nhs_111_live_triage":
        return nhs_111_live_triage(arguments)   # <--- ADD THIS LINE
    else:
        return f"[Error: Unknown tool '{tool_name}']"


In [ ]:
def agent():
    import json, time
    from openai import RateLimitError

    # --- SESSION MEMORY ---
    conversation_history = []
    user_profile = {}

    onboarding_active = False
    onboarding_spec = None

    triage_active = False
    triage_known_answers = {}

    print(intro_prompt + "\n")
    system_prompt = build_system_prompt(user_profile)

    print("You can continue asking questions now. Type 'exit' to stop.\n")

    HISTORY_WINDOW = 6
    MAX_OUT = 250
    MAX_TOOL_ROUNDS = 4
    MAX_RETRIES = 2

    # -----------------------------
    # SAFE MODEL CALL (TPM-aware)
    # -----------------------------
    def safe_create(**kwargs):
        for attempt in range(MAX_RETRIES + 1):
            try:
                return client.responses.create(**kwargs)
            except RateLimitError:
                if "input" in kwargs and isinstance(kwargs["input"], list):
                    sys_and_pins = [x for x in kwargs["input"] if x.get("role") == "system"]
                    others = [x for x in kwargs["input"] if x.get("role") != "system"]
                    kwargs["input"] = sys_and_pins + others[-3:]
                kwargs["max_output_tokens"] = min(kwargs.get("max_output_tokens", MAX_OUT), 150)
                time.sleep(0.2)
        raise

    # -----------------------------
    # MAIN LOOP
    # -----------------------------
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit", "stop"]:
            print("👋 Goodbye! Stay healthy.")
            break

        conversation_history.append({"role": "user", "content": user_input})

        # -------------------------------
        # PINNED CONTEXT (short!)
        # -------------------------------
        pinned = []

        if onboarding_active and onboarding_spec is not None:
            pinned.append({
                "role": "system",
                "content": (
                    "ONBOARDING MODE IS ACTIVE. "
                    "Ask the next onboarding question verbatim, in order. "
                    "Do NOT start triage or search during onboarding."
                )
            })

        if triage_active:
            pinned.append({
                "role": "system",
                "content": (
                    "TRIAGE MODE IS ACTIVE. "
                    "Do NOT call onboarding unless user explicitly says 'onboarding'. "
                    "Ask only triage follow-up questions until triage status='final'."
                )
            })

        # -------------------------------
        # FIRST MODEL CALL
        # -------------------------------
        resp = safe_create(
            model="gpt-4o-mini",
            store=True,
            input=[
                {"role": "system", "content": system_prompt},
                *pinned,
                *conversation_history[-HISTORY_WINDOW:]
            ],
            tools=tools,
            tool_choice="auto",
            max_output_tokens=MAX_OUT
        )

        final_response = resp
        triage_called_this_turn = False
        tool_rounds = 0

        # NEW: track whether we bailed out with unresolved tool calls
        bailed_with_unresolved_calls = False

        # -------------------------------
        # BATCH TOOL HANDLING LOOP
        # -------------------------------
        while True:
            tool_rounds += 1
            if tool_rounds > MAX_TOOL_ROUNDS:
                # We are stopping early; unresolved calls may remain.
                bailed_with_unresolved_calls = True
                break

            tool_calls = [item for item in final_response.output if item.type == "function_call"]
            if not tool_calls:
                break

            # Guard: only one triage call per user turn
            if (
                triage_called_this_turn
                and all(call.name == "nhs_111_live_triage" for call in tool_calls)
            ):
                # We are intentionally leaving these calls unresolved for this turn.
                bailed_with_unresolved_calls = True
                break

            outputs = [{"role": "system", "content": system_prompt}]

            for call in tool_calls:
                tool_name = call.name
                call_id = call.call_id
                raw_args = call.arguments

                if isinstance(raw_args, str):
                    try:
                        args = json.loads(raw_args)
                    except:
                        args = {}
                else:
                    args = raw_args or {}

                tool_result = execute_tool(tool_name, args)

                if tool_name == "nhs_111_live_triage":
                    triage_called_this_turn = True

                # onboarding state capture
                if isinstance(tool_result, dict) and tool_result.get("mode") == "llm_multiturn_onboarding":
                    onboarding_active = True
                    onboarding_spec = tool_result
                    triage_active = False

                # triage state capture (if tool returns status JSON)
                try:
                    parsed = tool_result if isinstance(tool_result, dict) else json.loads(tool_result)
                except:
                    parsed = None

                if isinstance(parsed, dict):
                    if parsed.get("status") == "need_more_info":
                        triage_active = True
                        triage_known_answers.update(parsed.get("known_answers_update", {}))
                    elif parsed.get("status") == "final":
                        triage_active = False

                tool_output_str = tool_result if isinstance(tool_result, str) else json.dumps(tool_result)

                outputs.append({
                    "type": "function_call_output",
                    "call_id": call_id,
                    "output": tool_output_str
                })

            final_response = safe_create(
                model="gpt-4o-mini",
                previous_response_id=final_response.id,
                input=outputs,
                tools=tools,
                tool_choice="auto",
                max_output_tokens=MAX_OUT
            )

        # -------------------------------
        # FINAL TEXT RESPONSE
        # -------------------------------
        agent_reply = final_response.output_text or ""

        # -------------------------------
        # 🔥 If we bailed with unresolved calls,
        # do NOT use previous_response_id.
        # Force a fresh text-only reply.
        # -------------------------------
        if bailed_with_unresolved_calls:
            forced = safe_create(
                model="gpt-4o-mini",
                store=True,
                input=[
                    {"role": "system", "content": system_prompt},
                    *pinned,
                    *conversation_history[-HISTORY_WINDOW:],
                    {
                        "role": "system",
                        "content": (
                            "You MUST respond to the user now in plain text. "
                            "Do NOT call any tools. "
                            "If triage is incomplete, ask the next 1–3 triage follow-up questions. "
                            "If triage is complete, give routing and next steps."
                        )
                    }
                ],
                tools=tools,
                tool_choice="none",
                max_output_tokens=200
            )
            agent_reply = forced.output_text or ""

        # -------------------------------
        # 🔥 Blank-response fix (safe only if no unresolved tools)
        # -------------------------------
        elif agent_reply.strip() == "":
            forced = safe_create(
                model="gpt-4o-mini",
                previous_response_id=final_response.id,
                input=[
                    {
                        "role": "system",
                        "content": (
                            system_prompt
                            + "\n\nYou MUST respond to the user now in plain text. "
                              "Do NOT call any tools. "
                              "If triage is incomplete, ask the next 1–3 triage follow-up questions. "
                              "If triage is complete, give routing and next steps."
                        )
                    }
                ],
                tools=tools,
                tool_choice="none",
                max_output_tokens=200
            )
            agent_reply = forced.output_text or ""

        clean_reply = strip_profile_tag(agent_reply)

        print("\nAssistant:", clean_reply, "\n")
        conversation_history.append({"role": "assistant", "content": clean_reply})

        # -------------------------------
        # PROFILE EXTRACTION
        # -------------------------------
        maybe_profile = extract_profile(agent_reply)
        if maybe_profile:
            user_profile = maybe_profile
            system_prompt = build_system_prompt(user_profile)

            onboarding_active = False
            onboarding_spec = None
            triage_active = False
            triage_known_answers = {}

            conversation_history.append({
                "role": "system",
                "content": f"Updated user profile for memory:\n{user_profile}"
            })


# Run Agent

In [ ]:

agent()

Hi there, welcome to London and to the LBS Community! My name is Evi - Your LBS Healthcare Companion.

Now that you’ve made it to London, I’m sure you have a lot of questions about navigating the NHS and LBS wellbeing services.
Feel free to start with one of the examples below to get you oriented.

- Better understand when and how to use all the services provided by the NHS (GP, NHS 111, A&E, and more!)
- Locate mental health or wellbeing support
- Get more information about preventative-care guidance

Or, type “onboarding” at any time, and I will ask a few brief questions to get to know you better.

You can continue asking questions now. Type 'exit' to stop.

You: How do we register with a GP?

Assistant: To register with a GP (General Practitioner) in the UK, you can follow these steps:

1. **Find a GP Practice**: Use the NHS website to search for local GP practices that are accepting new patients.
2. **Check Eligibility**: Ensure you are eligible to register with the practice; gener